<a href="https://colab.research.google.com/github/naolia1211/machine_learning_Algorithms/blob/main/B%E1%BA%A3n_sao_c%E1%BB%A7a_CO_Homogeneous_Ensemble_on__Malware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import trange

from sklearn.metrics import mean_squared_error, accuracy_score, auc, confusion_matrix, recall_score, precision_score, f1_score, classification_report

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import optimizers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/KDD/KDDTrain+.txt')
test_data = pd.read_csv('/content/drive/MyDrive/KDD/KDDTest+.txt')

In [ ]:
header = ['duration', 'protocol_type', 'service',
          'src_bytes', 'dst_bytes', 'flag', 'land',
          'wrong_fragment', 'urgent', 'hot', 'num_failed_logins',
          'logged_in', 'num_compromised', 'root_shell',
          'su_attempted', 'num_root', 'num_file_creations',
          'num_shells', 'num_access_files', 'num_outbound_cmds',
          'is_hot_login', 'is_guest_login', 'count', 'serror_rate',
          'rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_count',
          'srv_serror_rate', 'srv_rerror_rate', 'srv_diff_host_rate', 'dst_host_count',
          'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
          'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
          'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
          'dst_host_srv_rerror_rate', 'class', 'servere_points']

train_data.columns = header
test_data.columns = header

In [ ]:
print(train_data.shape, test_data.shape)
print(test_data.shape[0]/train_data.shape[0]*100)

(125972, 43) (22543, 43)
17.895246562728225


In [ ]:
def cal_accuracy(y_test, y_pred):

	print("Accuracy : \n",
	accuracy_score(y_test,y_pred)*100)

	print("Precision: \n",
	precision_score(y_test, y_pred)*100)

	print("Recall: \n",
	recall_score(y_test,y_pred)*100)

	print("F1 : \n",
	f1_score(y_test,y_pred)*100)

	'''fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
	print("AUC : \n",
	auc(y_test, y_pred)) '''

	print("Report : \n",
	classification_report(y_test, y_pred))

	print("Confusion Matrix: \n",
		confusion_matrix(y_test, y_pred))

In [ ]:
def dataPrep(data, label, drops):
  # We exclude all numeric columns
  num_cols = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  
  noEncode = data.pop(label)

  data.drop(drops, axis = 1, inplace=True)

  #print(data)

  data_catcol = data.select_dtypes(exclude=num_cols)
  data_numcols = data.select_dtypes(include=num_cols)
  # Steps to one-hot encoding:
  # We iterate through each categorical column name
  # Create encoded variables for each categorical columns
  # Concatenate the encoded variables to the DataFrame
  # Remove the original categorical variable
  for col in data_catcol.columns.values:
    one_hot_encoded_variables = pd.get_dummies(data_catcol[col],prefix=col)
    data_catcol = pd.concat([data_catcol,one_hot_encoded_variables],axis=1)
    data_catcol.drop([col],axis=1, inplace=True)

  noEncode = pd.Series(map(lambda x: 1 if x == 'normal' else 0, noEncode), name='class')
  
  data = pd.concat([data_numcols, data_catcol, noEncode], axis=1)

  return data

train_data = dataPrep(train_data, 'class', ['servere_points'])
test_data = dataPrep(test_data, 'class', ['servere_points'])

In [ ]:
for column in train_data.columns:
  if column not in test_data:
    test_data[column] = np.zeros(len(test_data))

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [ ]:
X_test = test_data.copy()  
Y_test = X_test.pop('class')
print(X_test.shape)
print(Y_test.shape)

(22543, 122)
(22543,)


In [ ]:
ensemble = 100
frac = 0.7

predictions_total = np.zeros(22543, dtype=float)

for i in range(ensemble):
    print("number of iteration:", i)
    print("predictions_total", predictions_total)

    # Sample randomly the train data
    Traindata = train_data.sample(frac=frac)
    X_train = Traindata.copy()  
    Y_train = X_train.pop('class') 
    
    model = Sequential()
    # Adding the input layer and the first hidden layer
    model.add(Dense(units=36, kernel_initializer = 'normal', activation = 'relu', input_dim = 122))
    # Adding the second hidden layer
    model.add(Dense(units = 24, kernel_initializer = 'normal', activation = 'relu'))
    # Adding the third hidden layer
    model.add(Dense(units = 16, kernel_initializer = 'normal', activation = 'relu'))
    # Adding the output layer
    model.add(Dense(units = 1, kernel_initializer = 'normal', activation = 'relu'))
    # Compiling the ANN
    adam = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    model.compile(loss='mse', optimizer=adam, metrics=['mean_squared_error'])
    # Fitting the ANN to the Training set
    model.fit(X_train, Y_train, batch_size = 512, epochs = 100, verbose = 0)

    # We use predict() to predict our values
    model_predictions = model.predict(X_test)
    
    model_predictions = model_predictions.flatten()
    print("TEST MSE for individual model: ", mean_squared_error(Y_test, model_predictions))
    print("")
    print(model_predictions)
    print("")

    model.save(f'/content/drive/MyDrive/Ensemble model/CO_Homogeneous_Ensemble/models/model_{i}')

    predictions_total = np.add(predictions_total, model_predictions)
 



number of iteration: 0
predictions_total [0. 0. 0. ... 0. 0. 0.]
705/705 [==============================] - 1s 1ms/step
TEST MSE for individual model:  0.24791635477364032

[0.         0.9640348  0.91531885 ... 0.9640348  0.9640348  0.8399373 ]

number of iteration: 1
predictions_total [0.         0.9640348  0.91531885 ... 0.9640348  0.9640348  0.83993733]
705/705 [==============================] - 1s 1ms/step
TEST MSE for individual model:  0.43413837026079527

[0. 0. 0. ... 0. 0. 0.]

number of iteration: 2
predictions_total [0.         0.9640348  0.91531885 ... 0.9640348  0.9640348  0.83993733]
705/705 [==============================] - 1s 1ms/step
TEST MSE for individual model:  0.17225936861795502

[0.        0.        0.        ... 0.        0.9903156 0.       ]

number of iteration: 3
predictions_total [0.         0.9640348  0.91531885 ... 0.9640348  1.95435041 0.83993733]
705/705 [==============================] - 1s 1ms/step
TEST MSE for individual model:  0.20062334918697222


In [ ]:
ensemble = 100

predictions_total = np.zeros(22543, dtype=float)

for i in trange(ensemble):

    ############################################################
    
    model = load_model(f'/content/drive/MyDrive/Ensemble model/CO_Homogeneous_Ensemble/models/model_{i}')

    ############################################################
    
    # We use predict() to predict our values
    model_predictions = model.predict(X_test)
    
    model_predictions = model_predictions.flatten()

    predictions_total = np.add(predictions_total, model_predictions)
 



number of iteration: 0
predictions_total [0. 0. 0. ... 0. 0. 0.]
705/705 [==============================] - 2s 2ms/step
TEST MSE for individual model:  0.24791635477364032

[0.         0.9640348  0.91531885 ... 0.9640348  0.9640348  0.8399373 ]

number of iteration: 1
predictions_total [0.         0.9640348  0.91531885 ... 0.9640348  0.9640348  0.83993733]
705/705 [==============================] - 1s 1ms/step
TEST MSE for individual model:  0.43413837026079527

[0. 0. 0. ... 0. 0. 0.]

number of iteration: 2
predictions_total [0.         0.9640348  0.91531885 ... 0.9640348  0.9640348  0.83993733]
705/705 [==============================] - 1s 1ms/step
TEST MSE for individual model:  0.17225936861795502

[0.        0.        0.        ... 0.        0.9903156 0.       ]

number of iteration: 3
predictions_total [0.         0.9640348  0.91531885 ... 0.9640348  1.95435041 0.83993733]
705/705 [==============================] - 1s 1ms/step
TEST MSE for individual model:  0.20062334918697222


In [ ]:
# out of for...loop
print("")
predictions_total = predictions_total/ensemble
print("MSE after ensemble: ", mean_squared_error(np.array(Y_test), predictions_total))
print("")
print(predictions_total)


MSE after ensemble:  0.14488281426702404

[0.         0.57219437 0.17113095 ... 0.39411771 0.74750806 0.12415159]


In [ ]:
y_pred = list(map(lambda x: 1 if x > 0.5 else 0, predictions_total))
cal_accuracy(Y_test, y_pred)

Accuracy : 
 81.61291753537684
Precision: 
 71.91683729721215
Recall: 
 94.03768921841211
F1 : 
 81.50296755767772
Report : 
               precision    recall  f1-score   support

           0       0.94      0.72      0.82     12832
           1       0.72      0.94      0.82      9711

    accuracy                           0.82     22543
   macro avg       0.83      0.83      0.82     22543
weighted avg       0.85      0.82      0.82     22543

Confusion Matrix: 
 [[9266 3566]
 [ 579 9132]]


In [ ]:
print(predictions_total.shape)

(22543,)
